# RP Phase 3
## Advanced Evaluation of ML Classification Models
## RP Group 1

While the paper focuses on binary classification problems, our problem is multi-label, with over 200 output classes, where each input record can belong to more than one output class. So, implementing the paper's methodology as-is will not work for the whole problem. A close analog to the paper's method is to analyze the same performance metrics and model behavior, but on one output class at a time.

So, each class will have its own results and plots: its own set of precision/recall curves, its own empirical risk curves, its own model behavior interpretations and so on. In this notebook, we only focus on getting these results for the first output class. Similar work can be donw for the other 200 or so classes.

### Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import make_multilabel_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification

## The Data

We use pickle so that we can re-use the same train/test split that we originally trained the two models on.

In [ ]:
import pickle

with open("X_train.pickle", "rb") as f1, open("X_test.pickle", "rb") as f2, \
    open("y_train.pickle", "rb") as f1_y, open("y_test.pickle", "rb") as f2_y:
    X_train = pickle.load(f1)
    X_test = pickle.load(f2)
    y_train = pickle.load(f1_y)
    y_test = pickle.load(f2_y)

## The Models

The only two models we have that were trained on the same split are a Random Forest model and a Linear SVM. Re-traninig the other models would take days to finish, given the size of our dataset and the large number of output classes of our multi-label problem.

We have previously-saved pickle files of these models, hoping we can re-use them. However to our surprise, the Random Forest file is awfully large (4.5 GB), and we are unable to load it into memory.

As most of the work requires comparing performance/behavior of multiple models, we will only show code we would have used here, without output sadly.

### Random Forest

In [ ]:
#file too large (uncompressed is 4.5 GB), unable to load into memory
#even tried to compress it, but load() de-compresses it and we ge the same problem
from joblib import load

rf_clf = load("clf_rf.joblib.gz")

In [ ]:
rf_clf

In [ ]:
rf_clf.predict_proba(X_test)[:5]

### Linear SVM

In [ ]:
with open("clf.pickle", "rb") as f:
    svm_clf = pickle.load(f)

In [ ]:
svm_clf.predict_proba(X_test)[:5]

## Performance Metrics

Notice that from now on, we only look at the first column of the predictions matrix of X_test. This is to only focus on the first output class, its results, plots, and model behavior for this particular class. 

### a- Traditional Metrics

We first assess model performance using some traditional metrics: precision, recall, and ROC curves. For each model, these metrics are computed, precision/recall curves are plotted, same for ROC curves.

However, these metrics alone are not enough for assessing the quality of probabilities, since these metrics only look at the quality of cassification splits.

In [ ]:
#traditional metrics: Precision, Recall, ROC curves
#on binary predictions, not probabilistic

#TODO

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

#only look at predictions of the first output class (notice the [:, 0] => first column only)
predictions = {
    'Random Forest': np.array(rf_clf.predict(X_test))[:, 0],
    'SVM': np.array(svm_clf.predict(X_test))[:, 0]
}

precisions = {k: precision_score(y_true=y_test, y_pred=v) for k, v in predictions.items()}
recalls = {k: recall_score(y_true=y_test, y_pred=v) for k, v in predictions.items()}

print(precisions)
print(recalls)

#ROC curves
from sklearn.metrics import plot_roc_curve

fig, ax = plt.subplots(1, figsize=(8, 6))
models = [rf_clf, svm_clf]
for i in range(len(models)):
    plot_roc_curve(models[i], X_test, y_test, ax=ax, name=list(predictions.keys())[i])
    
plt.title("ROC Curves")

In [ ]:
#plotting precision/recall results
from matplotlib import pyplot as plt

modelNames = list(precisions.keys())

fig, ax = plt.subplots(1, 2, figsize=(15, 6))

ax[0].barh(np.arange(len(modelNames)), precisions.values(),
                     align='center',
                     height=0.5,
                     tick_label=modelNames)

ax[0].set_title("Precision")

ax[1].barh(np.arange(len(modelNames)), recalls.values(),
                     align='center',
                     height=0.5,
                     tick_label=modelNames)

ax[1].set_title("Recall")

plt.show()

### b- Empirical Risk Curve

This is a metric that does reveal the quality of probabilistic predictions and observation rankings.

In [ ]:
#empirical risk curve for each model
#on probabilistic predictions
import copy

# predictions_proba = {
#     'Random Forest': [x[1] for x in rf_clf.predict_proba(X_test)],
#     'SVM': [x[1] for x in svm_clf.predict_proba(X_test)]
# }

# also here, we only focus on the first column of results
# (predict_proba returns a 3d matrix, so we slice it this way)
predictions_proba = {
    'Random Forest': [x[1] for x in [y[0] for y in rf_clf.predict_proba(X_test)]],
    'SVM': [x[1] for x in [y[0] for y in svm_clf.predict_proba(X_test)]]
}

def empirical_risk(l):
    l_pos = copy.deepcopy(l)
    l_pos = [(i, l_pos[i]) for i in range(len(l_pos))]
    l_pos = sorted(l_pos, key=lambda x: x[1])
    
    nbins=10
    items_per_bin = len(l_pos) // nbins
    
    res = []
    for i in range(nbins):
        lsub = l_pos[i*items_per_bin: (i+1)*items_per_bin]
        res += [sum([y_test[idx] for idx, _ in lsub])/items_per_bin]
    
    return res

empirical_risks = {k: empirical_risk(list(v)) for k, v in predictions_proba.items()}

plt.figure(figsize=(15, 6))

for k, v in empirical_risks.items():
    plt.plot([1, 2, 3, 4 ,5, 6, 7, 8, 9, 10], v, label=k)
    
plt.legend()
plt.title("Empirical Risk Curves")
plt.show()


### c- Precision/Recall at Top-k

In [ ]:
#precision and recall at top k, for varying values of k

ranked_lists = {k: sorted([(i, v[i]) for i in range(len(v))], key=lambda x: x[1], reverse=True) \
                for k, v in predictions_proba.items()}

plt.figure()
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

k_values = [5, 10] + list(range(20, len(y_test), 20))

for model, ranked_list in ranked_lists.items():
    # (the [y_test[x[0]] for x in ranked_lists[:k]] part is the ground truth at top-k)
    precisions = [sum([y_test[x[0]] for x in ranked_list[:k]])/k for k in k_values]
    recalls = [sum([y_test[x[0]] for x in ranked_list[:k]])/sum(y_test) for k in k_values]
    
    ax[0].plot(k_values, precisions, label=model)
    ax[1].plot(k_values, recalls, label=model)
    
    
plt.legend()

## Model Interpretation

### a- Feature Importance

To better understand model behavior, it is important to see which features each model relies the most on. FOr this, feature importance is computed for each feature and model, and the top 5 most important features for each model is stored.

In [ ]:
#get top 5 most important features from each model
#see if they are relevant

#feature importances of each model

feature_imp = {
    "Random Forest": rf_clf.best_estimator_.feature_importances_,
    "SVM": np.std(X_train, 0)*svm_clf.best_estimator_.coef_[0]
}

def getTopFiveFeatures(importances):
    #column names after the transforms
    colnames = range(1, len(importances)+1)
    
    importances_names = [(colnames[i], importances[i]) for i in range(len(importances))]
    
    importances_names = sorted(importances_names, key=lambda x: x[1], reverse=True)
    
    return [x[0] for x in importances_names[:5]]

print({k: getTopFiveFeatures(list(v)) for k, v in feature_imp.items()})


### b- Mistake Patterns

We now shift our attention to possible mistake patterns the models may have, more precisely the top-5 mistake patterns of each model. This helps us get a better understanding of where the models are being confused the most, and possibly act on these problematic patterns (and/or the model itself) to improve future model performance.

In [ ]:
#find all frequent patterns using FP-growth technique
#and then get the top-R "mistake patterns", i.e. those R patterns with the highest proba of mistake

from fp_growth import find_frequent_itemsets

X_test_with_idx = [[i] + X_test[i] for i in range(len(X_test))]

models = [rf_clf, svm_clf, dt_clf, lr_clf, ada_boost_clf]

#frequent patterns in data
frequent_patterns = find_frequent_itemsets(X_test_with_idx, 0.1)
nb_freq_patterns = sum(1 for _ in frequent_patterns)

for model in models:
    
    print("Results for model", model.best_estimator_)
    
    mistake_rates = [(idx, get_accuracy(model, frequent_patterns[idx])) for idx in range(nb_freq_patterns)]
    
    print(sorted(mistake_rates, key=lambda x: x[1], reverse=True))
    
def get_accuracy(model, pattern):
    y_true = [y_test[x[0]] for x in pattern]
    y_pred = model.predict(pattern[:, 1:])
    return accuracy_score(y_true=y_true, y_pred=y_pred)

In [ ]:
frequent_patterns = find_frequent_itemsets(X_test, 0.1)
sum(1 for _ in frequent_patterns)

### c- Model Comparison

Two models that output the same results are definitely redundant. This is why we are interested in comparing the ranked results returned by each of the models, to see how (dis)similar two models are.

For this, we plot Jaccard Similarities at top-k for each pair of models, just like the paper did. We use our test data and its predictions for these plots.

In [ ]:
#compare the results of the top-k highest risk of rejection applicants
#see how (dis)similar the result sets of the models are
#use Jaccard similarity at top-k

# For sets:
# | operator is union
# & operator is intersection
from sklearn.metrics import jaccard_score

def jaccard_similarity(l1, l2):
    if len(set(l1) | (set(l2))) == 0:
        return 0
    return len(set(l1) & (set(l2))) / len(set(l1) | (set(l2)))

ranked_lists = {k: sorted([(i, v[i]) for i in range(len(v))], key=lambda x: x[1], reverse=True) \
                for k, v in predictions_proba.items()}

plt.figure()
fig, ax = plt.subplots(1, figsize=(15, 6))

k_values = [5, 10] + list(range(20, len(y_test), 20))

i = 0
for model1, ranked_list1 in ranked_lists.items():
    for model2, ranked_list2 in list(ranked_lists.items())[i:]:
        if model1 != model2:
            
            plt.plot(k_values, [jaccard_similarity([x[0] for x in ranked_list1[:k]], \
                                              [x[0] for x in ranked_list2[:k]], \
                                              )
                               for k in k_values], label=f"{model1} - {model2}")
            
    i += 1
    
    
plt.legend()

In [ ]:
#number of children of FP-tree (corresponds to n of sets found)
fp_tree.root.children.keys()